In [1]:
import sys; sys.path.append("../src1")
from direct import Direct
import numpy as np, pygmo as pg, matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# boundary conditions
x0 = [0, 0, np.pi, 0]
xf = [1, 0, 0, 0]

In [ ]:
# duration boundsdis
Tlb, Tub = 3, 50

In [ ]:
# direct segment
seg = Direct(x0, xf, Tlb, Tub)

In [ ]:
# solve problem with linear guess
dv = seg.solve(seg.linear_guess(20, T=20), obj='time')

In [ ]:
# plot
times, h, states, controls = seg.decode(dv)
ax = seg.plot_traj(states, interp=True)
ax = seg.plot_timeline(times, states, controls, interp=True)

In [ ]:
dv = seg.solve(seg.interpolate(dv, 40))

In [ ]:
# plot
times, h, states, controls = seg.decode(dv)
ax = seg.plot_traj(states, interp=True)
ax = seg.plot_timeline(times, states, controls, interp=True)

In [ ]:
seg.decode(dv)

In [ ]:
def LGL(t0, tf, M):
    _, tau = np.polynomial.legendre.leggauss(M)
    return np.array([
        ((tf-t0)*tau[i]/M + 2*t0 + (tf-t0)*(2*i-1)/M)/2
        for i in range(M)
    ])

In [ ]:
LGL(0, 15, 20)